## Dependencies

In [15]:
import pandas as pd 
import numpy as np
from dotenv import load_dotenv
from sklearn.preprocessing import StandardScaler
import os
import requests
from datetime import datetime, timedelta
import re
import pandas_ta as ta
import warnings
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
pio.renderers.default = 'browser'
warnings.filterwarnings('ignore')

load_dotenv()

True

## Fetching and Merging Raw Data

In [16]:
class DataManager:
    
    def __init__(self):
        self.api_key = os.getenv('GLASSNODE_API_KEY')

    @staticmethod
    def datetime_to_unix(date):
        """Convert a datetime object to a Unix timestamp."""
        return int(date.timestamp())

    def _fetch_glassnode_data(self, endpoint, start, end, frequency):
        params = {
            'a': 'BTC',
            's': self.datetime_to_unix(start),
            'u': self.datetime_to_unix(end),
            'i': frequency,
            'f': 'JSON',
            'api_key': self.api_key  
        }
        
        base_url = 'https://api.glassnode.com/v1/metrics'
        url = f"{base_url}/{endpoint}"
        name = re.search(r'/([^/]*)$', endpoint).group(1)

        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data)
            if 't' in df.columns:
                df['t'] = pd.to_datetime(df['t'], unit='s')
            df.rename(columns={'v': name}, inplace=True)
            return df
        else:
            raise Exception(f"Failed to fetch data: {response.status_code} - {response.text}")
    
    def _fetch_and_merge_glassnode_data(self, endpoints, start, end, frequency):
        data_frames = []

        for endpoint in endpoints:
            df = self._fetch_glassnode_data(endpoint, start, end, frequency)
            if df is not None and not df.empty:
                data_frames.append(df.set_index('t'))
            else:
                print(f"Failed to fetch data for endpoint: {endpoint}")

        if data_frames:
            merged_df = pd.concat(data_frames, axis=1, join='outer')
            merged_df.reset_index(inplace=True)
            merged_df.set_index('t', inplace=True)
            return merged_df
        else:
            return None
        
        
    
    def get_trigger_data(self, start, end, frequency='24h'):
        # Fetches data for the trigger points
        short_term_endpoints = [
            os.getenv('BTC_PRICE'),
            os.getenv('SSR'),
            os.getenv('SUPPLY_IN_PROFIT')
        ]
        return self._fetch_and_merge_glassnode_data(short_term_endpoints, start, end, frequency)

    def get_context_data(self, start, end, frequency='24h'):
        # Fetches data for context determination
        contextual_endpoints = [
            os.getenv('BTC_PRICE'),
            os.getenv('BTC_REALIZED_PRICE'),
            os.getenv('PUELL_MULTIPLE'),
            os.getenv('MVRV_Z_SCORE'),
            os.getenv('ENTITY_ADJ_NUPL'),
            os.getenv('ENTITY_ADJ_DORMANCY_FLOW'),
            os.getenv('SUPPLY_IN_PROFIT')
        ]
        return self._fetch_and_merge_glassnode_data(contextual_endpoints, start, end, frequency)
    
    


In [17]:
class TradingStrategy:

    def __init__(self, data_manager):
        self.data_manager = data_manager

    def compute_triggers(self, start, end):
        
        start = datetime.strptime(start, '%Y-%m-%d')
        end = datetime.strptime(end, '%Y-%m-%d')
        
        trigger_data = self.data_manager.get_trigger_data((start - timedelta(days=360)), end)
        
        trigger_data['rsi_ssr_smoothed'] = ta.ema(ta.rsi(trigger_data['ssr_oscillator'], length=14), length=30)
        trigger_data['rsi_ssr_smoothed_median'] = trigger_data['rsi_ssr_smoothed'].rolling(window=10).median()
        #trigger_data['pct_profit_sma30'] = trigger_data['profit_relative'].rolling(30 * 24).mean()
        #trigger_data['acc_dacc_pct_profit'] = trigger_data['pct_profit_sma30'].pct_change(30 * 24)
        #trigger_data['pct_profit_mean'] = trigger_data['profit_relative'].expanding().mean()
        #trigger_data['pct_profit_sma360'] = trigger_data['profit_relative'].rolling(360 * 24).mean()
        #trigger_data['profit_std_pos'] = trigger_data['pct_profit_mean']+trigger_data['profit_relative'].expanding().std()
        #trigger_data['profit_std_neg'] = trigger_data['pct_profit_mean']-trigger_data['profit_relative'].expanding().std()

        trigger_data.drop(columns=['ssr_oscillator', 'profit_relative'], inplace=True)
        trigger_data = trigger_data[trigger_data.index >= start]
        
        return trigger_data

    def compute_context(self, start, end):

        start = datetime.strptime(start, '%Y-%m-%d')
        end = datetime.strptime(end, '%Y-%m-%d')
        
        context_data = self.data_manager.get_context_data((start - timedelta(days=360)), end)

        # Market condition calculations
        context_data['28d_mkt_gradient'] = (context_data['price_usd_close'].diff(28) - context_data['price_realized_usd'].diff(28) - 
                                            (context_data['price_usd_close'].diff(28) - context_data['price_realized_usd'].diff(28)).expanding().mean()) / \
                                            (context_data['price_usd_close'].diff(28) - context_data['price_realized_usd'].diff(28)).expanding().std()
        
        
        context_data['mayer_multiple'] = context_data['price_usd_close'] / context_data['price_usd_close'].rolling(200).mean()
        context_data['price_profit_corr'] = context_data['price_usd_close'].rolling(7).corr(context_data['profit_relative'])

    

        # Detect market tops and bottoms
        context_data['top_detection'] = (np.where(context_data['mvrv_z_score'] > 3.8, 1, 0) * 
                                         np.where(context_data['mayer_multiple'] >= 1.3, 1, 0) *
                                         np.where(context_data['net_unrealized_profit_loss_account_based'] >= 0.6, 1, 0) *
                                         np.where(context_data['28d_mkt_gradient'] >= 7, 1, 0)) * context_data['price_usd_close']
        
        context_data['bottom_detection'] = (np.where(context_data['mvrv_z_score'] <= 0, 1, 0) * 
                                            np.where(context_data['mayer_multiple'] <= 0.8, 1, 0) *
                                            np.where(context_data['price_usd_close'] <= context_data['price_realized_usd'], 1, 0) *
                                            np.where(context_data['net_unrealized_profit_loss_account_based'] <= 0, 1, 0) *
                                            np.where(context_data['puell_multiple'] <= 0.5, 1, 0) *
                                            np.where(context_data['dormancy_flow'] <= 200000, 1, 0)) * context_data['price_usd_close']
        
        
    

        # Apply context determination
        context_data['context'] = context_data.apply(self.determine_context, axis=1)
        context_data['context'].fillna(method='ffill', inplace=True)
        context_data.drop(columns=['top_detection', 'bottom_detection'], inplace=True)
        context_data = context_data[context_data.index >= start]
        
        
        return context_data

    @staticmethod
    def determine_context(row):
        if row['bottom_detection'] != 0 and (row['top_detection'] == 0 or row.name < row.index[row['top_detection'] != 0].max()):
            return int(1)
        elif row['top_detection'] != 0 and (row['bottom_detection'] == 0 or row.name < row.index[row['bottom_detection'] != 0].max()):
            return int(0)
        else:
            return np.nan
    
    def merge_data(self, trigger_data, context_data):
        
        full_data=pd.merge_asof(trigger_data, context_data[['context']], on='t', direction='forward')
        full_data['context'] = full_data['context'].ffill()
        full_data.set_index('t', inplace=True)
        #full_data.dropna(axis=0, inplace=True)

        return full_data




In [18]:
class TradingEnvironment:
    def __init__(self, data, window_size, scaler=StandardScaler()):
        self.data = data
        self.window_size = window_size
        self.cash = 10000
        self.initial_cash = 10000  # Starting cash
        self.bitcoin = 0  # Bitcoin holding, positive for long, negative for short
        self.entry_price = 0  # Price at which the last trade was executed
        self.net_worth = self.cash  # Net worth initialized to starting cash
        self.scaler = scaler  # Optional scaler
        self.scaled_data = pd.DataFrame(scaler.fit_transform(self.data), columns=self.data.columns, index=self.data.index)
        self.current_step = 0
        self.position_state = 1  # 0: long, 1: cash, 2: short
        self.reset()

    def reset(self):
        self.current_step = self.window_size
        self.cash = 10000
        self.bitcoin = 0
        self.entry_price = 0
        self.net_worth = self.cash
        self.position_state = 1
        return self._get_state()

    def _get_state(self):
        state_columns = [col for col in self.scaled_data.columns if col != 'context']
        state = self.scaled_data[state_columns].iloc[self.current_step - self.window_size:self.current_step].values
        return state.astype(float)

    def step(self, action):
        if self.current_step + 1 >= len(self.data):
            raise IndexError("Attempted to step beyond the data boundary.")

        current_price = self.data.iloc[self.current_step]['price_usd_close']
        
        if action == 0:  # Long
            if self.position_state != 0 and self.cash > 0:
                self.bitcoin = self.cash / current_price
                self.entry_price = current_price
                self.cash = 0
                self.position_state = 0

        elif action == 1:  # Cash
            if self.position_state == 0:  # Closing long
                self.cash = self.bitcoin * current_price
                self.bitcoin = 0
            elif self.position_state == 2:  # Closing short
                profit = -self.bitcoin * (self.entry_price - current_price)
                self.cash += profit
                self.bitcoin = 0
            self.position_state = 1

        elif action == 2:  # Short
            if self.position_state != 2 and self.cash > 0:
                self.bitcoin = -self.cash / current_price  # Short position
                self.entry_price = current_price
                self.cash = 0
                self.position_state = 2

        self.current_step += 1
        self.net_worth = self.cash + (self.bitcoin * current_price if self.bitcoin > 0 else 0)
        
        # Calculate reward based on net worth change, not just final amount
        reward = self.net_worth - self.initial_cash
        
        if reward > 0:
            reward = 1
        elif reward < 0:
            reward = -1
        else:
            reward = 0

        done = self.net_worth <= 0 or self.current_step >= len(self.data) - 1

        return self._get_state(), reward, done


In [19]:
class DuelingQNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(DuelingQNetwork, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.fc1 = nn.Linear(state_size, 42)
        self.fc2 = nn.Linear(42, 42)
        
        # Value stream
        self.value_fc = nn.Linear(42, 1)
        
        # Advantage stream
        self.advantage_fc = nn.Linear(42, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        
        value = self.value_fc(x)
        advantage = self.advantage_fc(x)
        
        q_value = value + (advantage - advantage.mean(dim=1, keepdim=True))
        return q_value.squeeze(1)

class DuelingDQNAgent:
    def __init__(self, state_size, action_size):    
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def _build_model(self):
        return DuelingQNetwork(self.state_size, self.action_size)

    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state, context):
        if random.random() <= self.epsilon:
            if context == 1:
                return random.choice([0, 1])  # Long or Cash
            elif context == 0:
                return random.choice([1, 2])  # Cash or Short
            
        state = torch.FloatTensor(state).reshape(-1, self.state_size).unsqueeze(0)  # Flatten the state
        
        act_values = self.model(state) # returns Q-values 

        if context == 1:
            return torch.argmax(act_values[:, :2], 1).item()  # Only consider Long or Cash
        elif context == 0:
            return torch.argmax(act_values[:, 1:], 1).item() + 1  # Only consider Cash or Short

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return

        minibatch = random.sample(self.memory, batch_size)

        for state, action, reward, next_state, done in minibatch:
            
            state = torch.FloatTensor(state).reshape(-1, self.state_size)
            next_state = torch.FloatTensor(next_state).reshape(-1, self.state_size)
            
            current_qs = self.model(state)
            next_qs_target = self.target_model(next_state)
            
            max_next_qs = torch.max(next_qs_target, dim=1)[0]
            target_qs = current_qs.clone().detach()
            targets = reward + self.gamma * max_next_qs * (1 - done)  # Done flag to zero out terminal states
            
            target_qs[0, action] = targets
            
            loss = nn.MSELoss()(current_qs, target_qs)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        # Epsilon decay
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


    def load(self, name):
        self.model.load_state_dict(torch.load(name))

    def save(self, name):
        torch.save(self.model.state_dict(), name)





In [20]:
def train_dqn(agent, env, repetitions, batch_size):
    
    for e in range(repetitions):
        state = env.reset()
        returns = []
        steps = 0

        while True:
            context = env.data.iloc[env.current_step]['context']  # gets latest context
            action = agent.act(state, context)  # either 0, 1, or 2
            next_state, reward, done = env.step(action)  # acts and obtains a reward
            returns.append(env.net_worth/env.initial_cash-1)
            steps += 1
            
            agent.remember(state, action, reward, next_state, done)  # stores the experience

            state = next_state
            
            if done:
                agent.update_target_model()  # Periodically update the target model
                #print(f"Repetition {e+1} of {repetitions} - \n Net Worth: {env.net_worth} \n Cumulative Return: {100*(returns[steps-1])}%, Steps: {steps}")
                break

            # Only perform a replay if enough memory is available to sample from
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)

            # Optional: Output information periodically
            if steps % 1 == 0:
                print(f"Action: {action}| Date: {str(env.data.index[env.current_step])} | BTC Price: {env.data.iloc[env.current_step]['price_usd_close']} | Cumulative Return: {100*(returns[steps-1])}% | Step: {env.current_step} | Rep: {e} ")

    print("Training completed.")


In [21]:
data_manager = DataManager()
strategy = TradingStrategy(data_manager)

start = '2020-01-01'
end = '2023-06-22'

context_data = strategy.compute_context('2011-08-1', end)
trigger_data = strategy.compute_triggers(start, end)
full_data = strategy.merge_data(trigger_data, context_data)
env = TradingEnvironment(full_data, window_size=5) # must be mindful of window size (days)

In [22]:
env.reset()
state_size = env._get_state().shape[0] * env._get_state().shape[1]
action_size = 3  # Long, Cash, Short
agent = DuelingDQNAgent(state_size, action_size)

In [28]:
#train_dqn(agent=agent, env=env, repetitions=3, batch_size=32)
full_data

,price_usd_close,rsi_ssr_smoothed,rsi_ssr_smoothed_median,context
t,,,,
2020-01-01,7199.661101,51.319011,49.034666,1.0
2020-01-02,6985.659023,51.250481,49.739185,1.0
2020-01-03,7347.597378,51.783224,50.338955,1.0
2020-01-04,7415.202445,52.322456,50.747238,1.0
2020-01-05,7410.812541,52.855039,51.090999,1.0
...,...,...,...,...
2023-06-17,26517.594549,38.763996,38.726155,1.0
2023-06-18,26325.890181,39.007212,38.726155,1.0
2023-06-19,26783.247197,39.547575,38.726155,1.0


In [24]:
class Backtester:
    
    def __init__(self, environment, agent, initial_capital=10000):
        self.data = environment
        self.agent = agent
        self.window_size = environment.window_size
        self.initial_capital = initial_capital
        self.results = {}
        self.run_backtest()

    def run_backtest(self):
        self.data.data['position'] = 0
        self.cash = self.initial_capital
        self.bitcoin = 0
        self.net_worth = self.cash
        self.positions = []

        for i in range(self.window_size, len(self.data)):
            state = self.data.iloc[i-self.window_size:i].values
            context = self.data.iloc[i]['context']
            action = self.agent.act(state, context)

            current_price = self.data.iloc[i]['price_usd_close']

            if context == 1:
                if action == 0 and self.cash > 0:  # Long
                    self.bitcoin += self.cash / current_price
                    self.cash = 0
                    self.positions.append((self.data.index[i], 'long'))
                elif action == 1:  # Cash
                    pass
            elif context == 0:
                if action == 1 and self.cash > 0:  # Cash
                    pass
                elif action == 2 and self.bitcoin > 0:  # Short
                    self.cash += self.bitcoin * current_price
                    self.bitcoin = 0
                    self.positions.append((self.data.index[i], 'short'))

            self.net_worth = self.cash + self.bitcoin * current_price
            self.data.loc[self.data.index[i], 'net_worth'] = self.net_worth

        self.calculate_statistics()

    def calculate_statistics(self):
        self.results['Start'] = self.data.index.min()
        self.results['End'] = self.data.index.max()
        self.results['Duração'] = self.results['End'] - self.results['Start']
        self.results['Tempo de Exposição [%]'] = np.mean(self.data['position'] != 0) * 100
        self.results['Saldo Final [$]'] = self.data['net_worth'].iloc[-1]
        self.results['Pico de Valor [$]'] = self.data['net_worth'].max()
        self.results['Retorno AlfaTrader [%]'] = ((self.results['Saldo Final [$]'] / self.initial_capital - 1) * 100)
        self.results['Retorno Buy & Hold [%]'] = ((self.data['price_usd_close'].iloc[-1] / self.data['price_usd_close'].iloc[0] - 1) * 100)
        self.results['Retorno (Ann.) [%]'] = self.results['Retorno AlfaTrader [%]'] / (self.results['Duração'].days / 365.25)
        self.results['Volatilidade (Ann.) [%]'] = self.data['strategy_return'].std() * np.sqrt(365*24) * 100

        # Filter to include only non-zero returns or when the strategy is active
        active_returns = self.data['strategy_return'][self.data['strategy_return'] != 0]

        # Calculate Sharpe Ratio using non-zero returns
        if active_returns.empty:
            self.results['Sharpe Ratio'] = 0  # Handle case where there are no active returns
        else:
            self.results['Sharpe Ratio'] = (active_returns.mean() / self.data['strategy_return'].std()) * np.sqrt(8760)

        # Calculate negative returns for the downside deviation
        negative_returns = self.data['strategy_return'][self.data['strategy_return'] < 0]

        # Calculate Sortino Ratio using non-zero returns and downside deviation
        if not negative_returns.empty and not active_returns.empty:
            mean_active_returns = active_returns.mean()
            downside_deviation = negative_returns.std()
            annual_factor = np.sqrt(8760)  # Assuming 24/7 trading for crypto markets
            self.results['Sortino Ratio'] = (mean_active_returns / downside_deviation) * annual_factor
        else:
            self.results['Sortino Ratio'] = 0  # Handle case where there are no negative returns or active returns

        # Trading statistics
        trades = self.data['position'].diff().fillna(0) != 0
        self.results['# Trades'] = trades.sum()

    def get_results(self):
        return pd.DataFrame([self.results])




In [25]:
class StrategyVisualizer:
    def __init__(self, data):
        self.data = data

    def plot_results(self):
        # Create subplots: one for portfolio values and one for BTC price with signals
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                            vertical_spacing=0.1, subplot_titles=('Valor do Portfólio', 'Preço BTC com Sinais'))

        # Plotting portfolio values: Strategy vs Buy & Hold
        fig.add_trace(go.Scatter(x=self.data.index, y=self.data['strategy_cumulative_return'], name='AlfaTrader ®', line=dict(color='green')), row=1, col=1)
        #fig.add_trace(go.Scatter(x=self.data.index, y=self.data['price_usd_close'], name='BTC', line=dict(color='blue')), row=1, col=1)
        fig.add_trace(go.Scatter(x=self.data.index, y=self.data['market_portfolio_cumulative_return'], name='Comprar e Segurar', line=dict(color='blue')), row=1, col=1)
        # BTC Price and action markers
        fig.add_trace(go.Scatter(x=self.data.index, y=self.data['price_usd_close'], name='Preço BTC', line=dict(color='black')), row=2, col=1)

        # Adding markers only at points of action change
        action_changes = self.data[self.data['action'] != self.data['action'].shift(1)]
        buys = action_changes[action_changes['action'] == 'long']
        cash = action_changes[action_changes['action'] == 'cash']
        shorts = action_changes[action_changes['action'] == 'short']

        fig.add_trace(go.Scatter(x=buys.index, y=buys['price_usd_close'], mode='markers', marker=dict(color='#4AD811', size=8, symbol='triangle-up'), name='Compra'), row=2, col=1)
        fig.add_trace(go.Scatter(x=cash.index, y=cash['price_usd_close'], mode='markers', marker=dict(color='#FFCC2D', size=8, symbol='square'), name='Caixa'), row=2, col=1)
        fig.add_trace(go.Scatter(x=shorts.index, y=shorts['price_usd_close'], mode='markers', marker=dict(color='#C70039', size=8, symbol='triangle-down'), name='Venda'), row=2, col=1)

        # Update layout
        fig.update_layout(height=800, width=1000, title_text="", hovermode="x unified", plot_bgcolor='white', paper_bgcolor='white', font=dict(color='black'))
        fig.show()



    def plot_btc_with_signals(self):
        # Create a figure with custom subplots
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        # Plotting BTC price as a line graph
        fig.add_trace(go.Scatter(x=self.data.index, y=self.data['price_usd_close'], name='Preço BTC', line=dict(color='blue')), secondary_y=False)

        # Ensure that only points where detection is marked as '1' have non-zero heights
        # Top detections as green bars
        top_heights = self.data['price_usd_close'].where(self.data['top_detection'] != 0, 0)
        fig.add_trace(go.Bar(x=self.data.index, y=top_heights, name='Detecção de Topo', marker_color='green'), secondary_y=False)

        # Bottom detections as red bars
        bottom_heights = self.data['price_usd_close'].where(self.data['bottom_detection'] != 0, 0)
        fig.add_trace(go.Bar(x=self.data.index, y=bottom_heights, name='Detecção de Fundo', marker_color='red'), secondary_y=False)

        # Update layout for aesthetics and readability
        fig.update_layout(
            title='Preço BTC com Indicadores de Topo e Fundo',
            xaxis_title='',
            yaxis_title='Preço BTC',
            legend_title='Legenda',
            hovermode='x unified',
            plot_bgcolor='white',
            paper_bgcolor='white',
            font=dict(color='black')
        )

        # Adjust the y-axes visual appearance
        fig.update_yaxes(title_text="Preço BTC", secondary_y=False)

        fig.show()


In [26]:
visualizer = StrategyVisualizer(final_data)
#visualizer.plot_btc_with_signals()
visualizer.plot_results()


NameError: name 'final_data' is not defined